In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy.stats import norm
from collections import Counter
import numpy as np

In [24]:
class Read_Df(object):
    def __init__(self, idx, csv_path, plants_list):
        self.idx = idx
        self.csv_path = csv_path
        self.plant_name_str = plants_list[self.idx]
        
    def Read_csvs(self):
        csv_path_list = os.listdir(self.csv_path)
        filter_csv_list = [csv for csv in csv_path_list if csv.split('_')[1] == self.plant_name_str]
        return filter_csv_list
    
    def Add_numqk_perpng(self):
        name_list = []
        csv_list = self.Read_csvs()
        assert type(csv_list) == list
        for csv_name in csv_list:
            name1 = csv_name.split('_')[2]
            name_list.append(name1)
        stat_per_png = Counter(name_list)
        a = stat_per_png.items()
        d = {i[0]:i[1] for i in a}
        sorted_keys = sorted(d.keys())
        sorted_dict_by_keys = {key: d[key] for key in sorted_keys}
        df_s = pd.Series(sorted_dict_by_keys)
        df = pd.DataFrame(df_s, columns = ['Count'])
        return df
    
    def Concat_diffpng2onedf(self):
        # 把处于相同图片的气孔的参数加在一个df中
        # return: type = dict
        diff_df_dict = {}
        name_list = []
        csv_files = self.Read_csvs()
        for csv_name in csv_files:
            name1 = csv_name.split('_')[2]
            name_list.append(name1)
        unqiue_name_count = Counter(name_list).items()
        unqiue_name = {name[0]:[] for name in unqiue_name_count}
        for csv_name in csv_files:
            name2 = csv_name.split('_')[2]
            if name2 in unqiue_name.keys():
                unqiue_name[name2].append(csv_name)
        assert unqiue_name.__len__() == self.Add_numqk_perpng().shape[0]
        for png_name, csves in unqiue_name.items():
            csv_path = [os.path.join(self.csv_path, i) for i in csves]
            df_list = [pd.read_csv(csv) for csv in csv_path]
            df = pd.concat(df_list, axis=0)
            diff_df_dict[png_name] = df
        return diff_df_dict
    
    def Concat_diffpng2one_add_perqk(self):
        diff_df_dict = self.reshape_dfto4col()
        df_qk_perpng =self.Add_numqk_perpng().T
        df_per_png = []
        assert diff_df_dict.__len__() == df_qk_perpng.shape[1]
        for df_name, df in diff_df_dict.items():
            df_stat = df.describe().iloc[1, :]
            df_stat = pd.DataFrame(df_stat).T
            df_stat['png_name'] = df_name
            df_stat['Count'] = df_qk_perpng[df_name].values
            df_per_png.append(df_stat)
            
        df_all_png = pd.concat(df_per_png, axis=0).reset_index()
        return df_all_png.iloc[:,1:]
    
    def Cacu_qk_num(self):
        num = len(self.Read_csvs())
        return num
    
    def __call__(self):
        assert self.reshape_dfto4col().shape[0] == self.Cacu_qk_num(), 'qk的总数与datafream的行数对不上。'
        
    def Add_class(self, str_class, save_path=None):
        # 提取后的参数，再进行SVM建模；
        assert type(str_class == str), 'str_class type must be str, this is {0}'.format(type(str_class))
        assert str_class == 'n' or '2n', 'str_class must in n or Dn'
        df = self.Add_Aspect_ratio()
        print(df.shape[0])
        if str_class == 'n':
            class_num = [1] * df.shape[0]
        elif str_class == '2n':
            class_num = [2] * df.shape[0]
        df_class = pd.DataFrame(list(map(int, class_num)))
        df_class.columns = ['labels']
        df_with_class = pd.concat([df, df_class], axis=1)
        if save_path:
            if os.path.exists(save_path) is False:
                os.makedirs(save_path, exist_ok=True)
            csv_name = self.plant_name_str + '.csv'
            png_path = os.path.join(save_path, csv_name)
            df_with_class.to_csv(png_path)
        return df_with_class
    
    def Add_Aspect_ratio(self):
        df = self.Concat_diffpng2one_add_perqk()
        df_long = df['long dia']
        df_short = df['short dia']
        df_asp = pd.DataFrame((df_long - df_short) / df_long)
        df_asp.columns = ['aspect']
        df_fivecalss = pd.concat([df, df_asp], axis = 1)
        return df_fivecalss
    
    def plot_bar(self, save_path = None):
        df = self.Add_Aspect_ratio()
        print(df.columns)
        fig, axes = plt.subplots(ncols=3, nrows=2, sharex=False, sharey=False)
        sns.distplot(df['area'], ax = axes[0, 0], color = '#ab1d22', fit=norm, kde=False, axlabel= 'Area (μm²)',
                     fit_kws={"color": "r", "lw": 1.4, "label": "fit_norm"},
                       hist_kws={"histtype": "bar", "linewidth": 1.4,
                                 "alpha": 0.3, "color": "b"})
        axes[0,0].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        sns.distplot(df['circumferfence'], ax=axes[0, 1], axlabel='Circumference (μm)',kde=False,fit = norm)
        axes[0,1].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        sns.distplot(df['short dia'], ax=axes[1, 0], axlabel='Short Diameter (μm)', kde=False, fit=norm)
        axes[1,0].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        sns.distplot(df['long dia'], ax=axes[1, 1], axlabel='Long Diameter (μm)', kde=False, fit=norm)
        axes[1,1].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        sns.distplot(df['aspect'], ax=axes[0, 2], axlabel='Aspect Ratio', kde=False, fit=norm)
        axes[0,2].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        
        sns.distplot(df['Count'], ax=axes[1, 2], axlabel='Count', kde=False, fit=norm)
        axes[1,2].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        fig.subplots_adjust(hspace=0.5, wspace=0.3)  # 调整hspace和wspace的数值来改变间隙大小
        plt.show()
        if save_path:
            if os.path.exists(save_path) is False:
                os.makedirs(save_path, exist_ok=True)
            name = self.plant_name_str + '.png'
            png_path = os.path.join(save_path,name)
            fig.savefig(png_path)
    
    def stat_df(self):
        df = self.Concat_diffpng2one_add_perqk()
        return df.describe()
       
    def reshape_dfto4col(self):
        df_dict = self.Concat_diffpng2onedf()
        reshape_df_dict = {}
        num = 0
        for df_name, df in df_dict.items():
            if df.shape[0] not in [2,4]:
                df_dia = df['diameter']
                df_are = df['area']
                df_circumference = df['approx_perimeter']
                df_diameter_long = df_dia[1]
                df_diameter_short = df_dia[0]
                df_a = df_are[0]
                df_c = df_circumference[0]
                print(df_c)
                print(df_name)
                df_all = np.stack([df_a, df_c, df_diameter_long, df_diameter_short], axis = 1) 
                df_all = pd.DataFrame(df_all)
                df_all.columns = ['area', 'circumferfence', 'short dia', 'long dia']
            else:
                df_dia = df['diameter']
                df_are = df['area']
                df_circumference = df['approx_perimeter']
                df_diameter_long = df_dia[1]
                df_diameter_short = df_dia[0]
                df_a = df_are[0]
                df_c = df_circumference[0]
                df_all = np.array([df_a, df_c, df_diameter_short, df_diameter_long])
                df_all = pd.DataFrame(df_all).T
                df_all.columns = ['area', 'circumferfence', 'short dia', 'long dia']
            reshape_df_dict[df_name] = df_all
            print(num)
            num += 1
            
        return reshape_df_dict

In [25]:
plants_list = os.listdir(r'D:\qk_maskrcnn_trs\qk_data\test_dataset')
def det_plant_name(plants_list, name):
    for i, pn in enumerate(plants_list):
        if pn == name:
            return i, plants_list
plants_list

['2n-train', 'n-train', 'others', 'r2n', 'reshape-2n', 'reshape-n', 'rn']

In [28]:
nu = det_plant_name(plants_list, 'reshape-2n')[0]
n = Read_Df(nu, r'D:\qk_maskrcnn_trs\edge_det\fit_ell_csv', plants_list)
n.Add_class(str_class='2n', save_path='./')

0    140.161347
0    129.213833
0    131.612034
Name: approx_perimeter, dtype: float64
reshape-2n1000
0
0    128.030177
0    138.641579
0    135.162976
Name: approx_perimeter, dtype: float64
reshape-2n1001
1
0    119.352120
0    112.260091
0    117.103654
Name: approx_perimeter, dtype: float64
reshape-2n1002
2
3
0    110.193031
0    125.540398
0    116.002582
0    126.077472
0    110.177713
0    124.072444
Name: approx_perimeter, dtype: float64
reshape-2n1004
4
5
6
7
8
9
0    120.565030
0    112.311732
0    111.516480
0     77.589106
Name: approx_perimeter, dtype: float64
reshape-2n100
10
11
12
13
0    124.678305
0    115.849533
0     72.666886
Name: approx_perimeter, dtype: float64
reshape-2n1013
14
0    116.863228
0    134.483214
0    124.446308
Name: approx_perimeter, dtype: float64
reshape-2n1014
15
16
0    138.013122
0    126.572610
0    143.644075
Name: approx_perimeter, dtype: float64
reshape-2n1016
17
0    110.313110
0     93.328462
0    106.681831
0    104.567579
0    101.3510

,area,circumferfence,short dia,long dia,png_name,Count,aspect,labels
0,336.644405,133.662405,24.937061,17.224704,reshape-2n1000,3,-0.447750,2
1,351.338365,133.944911,23.334511,19.183108,reshape-2n1001,3,-0.216409,2
2,261.950524,116.238621,20.744022,16.092115,reshape-2n1002,3,-0.289080,2
3,296.214465,123.137177,17.528113,21.549584,reshape-2n1003,2,0.186615,2
4,274.519306,118.677273,20.957787,16.672744,reshape-2n1004,6,-0.257009,2
...,...,...,...,...,...,...,...,...
961,305.759767,127.396211,23.669362,16.516437,reshape-2n997,3,-0.433079,2
962,324.871758,129.596429,17.583541,23.443508,reshape-2n998,2,0.249961,2
963,354.899897,134.240977,23.081768,19.565413,reshape-2n999,3,-0.179723,2
964,275.855188,119.138576,21.261741,16.509800,reshape-2n99,3,-0.287825,2


In [29]:
def exchage_cols(csv_path, save_path, nor2n):
    df = pd.read_csv(csv_path)
    df_aple0 = df[df['aspect']<0]
    df_aple0[['short dia', 'long dia']] = df[['long dia', 'short dia']]
    df.loc[df['aspect'] < 0, ['short dia', 'long dia']] = df_aple0[['short dia', 'long dia']]
    df['aspect'] = (df['long dia'] - df['short dia']) / df['long dia']
    df[df['aspect']<0]
    if save_path:
        if os.path.exists(save_path) is False:
            os.makedirs(save_path, exist_ok=True)
        file_name =  str(nor2n) + '.csv'
        file_path = os.path.join(save_path, file_name)
        df.to_csv(file_name)
    return df
df = exchage_cols(csv_path=r'./reshape-2n.csv', save_path='./csv', nor2n='2n')
df

C:\Users\16477\AppData\Local\Temp\ipykernel_13344\1706105.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aple0[['short dia', 'long dia']] = df[['long dia', 'short dia']]


,Unnamed: 0,area,circumferfence,short dia,long dia,png_name,Count,aspect,labels
0,0,336.644405,133.662405,17.224704,24.937061,reshape-2n1000,3,0.309273,2
1,1,351.338365,133.944911,19.183108,23.334511,reshape-2n1001,3,0.177908,2
2,2,261.950524,116.238621,16.092115,20.744022,reshape-2n1002,3,0.224253,2
3,3,296.214465,123.137177,17.528113,21.549584,reshape-2n1003,2,0.186615,2
4,4,274.519306,118.677273,16.672744,20.957787,reshape-2n1004,6,0.204461,2
...,...,...,...,...,...,...,...,...,...
961,961,305.759767,127.396211,16.516437,23.669362,reshape-2n997,3,0.302202,2
962,962,324.871758,129.596429,17.583541,23.443508,reshape-2n998,2,0.249961,2
963,963,354.899897,134.240977,19.565413,23.081768,reshape-2n999,3,0.152343,2
964,964,275.855188,119.138576,16.509800,21.261741,reshape-2n99,3,0.223497,2


In [32]:
#， 数据清洗
# 异常点检测，以及去异常值
import matplotlib.pyplot as plt
import seaborn as sns
dfn = pd.read_csv(r'./csv/n.csv').iloc[:,2:]
df2n = pd.read_csv(r'./csv/2n.csv')[2:].iloc[:,2:]
dfnn = pd.read_csv(r'./csv/n.csv').iloc[:,2:]
df2nn = pd.read_csv(r'./csv/2n.csv')[2:].iloc[:,2:]
print(dfn.shape)
print(df2n.shape)
df_list = [dfn, dfnn, df2n, df2nn]
for df in df_list:
    df.set_index('png_name', drop=True, inplace=True)

def plot_percol(column):
    plt.scatter(x=range(column.shape[0]), y=column)
    plt.show()

#dfn.apply(plot_percol)
#df2n.apply(plot_percol)
#df2n.apply(plot_percol)

(1729, 8)
(964, 8)


In [33]:
from sklearn.ensemble import IsolationForest
for col in dfn.columns:
    if col == 'png_name':
        continue
    clf = IsolationForest(contamination=0.1, n_estimators=100, max_samples='auto', max_features=1)
    clf.fit(dfn[[col]]) 
    dfn[col] = clf.predict(dfn[[col]]) != -1
print(dfn)  
df_n = dfnn[dfn]

df_n.dropna().to_csv('df_1_filter.csv')

               area  circumferfence  short dia  long dia  Count  aspect  \
png_name                                                                  
reshape-n1000  True            True       True      True   True    True   
reshape-n1001  True            True       True      True   True    True   
reshape-n1002  True            True       True      True   True    True   
reshape-n1003  True            True       True      True   True   False   
reshape-n1004  True            True       True      True   True    True   
...             ...             ...        ...       ...    ...     ...   
reshape-n997   True            True       True      True  False    True   
reshape-n998   True            True       True      True   True    True   
reshape-n999   True            True       True      True   True   False   
reshape-n99    True            True       True      True   True    True   
reshape-n9     True            True       True      True   True    True   

               labels  


In [34]:
for col in df2n.columns:
    if col == 'png_name':
        continue
    clf = IsolationForest(contamination=0.2, n_estimators=100, max_samples='auto', max_features=1)
    clf.fit(df2n[[col]]) 
    df2n[col] = clf.predict(df2n[[col]]) != -1

df_2n = df2nn[df2n]
df_2n.dropna().to_csv('df_2_filter.csv')

In [38]:
df1 = pd.read_csv(r'./df_1_filter.csv')
df2 = pd.read_csv(r'./df_2_filter.csv')
df1.shape

(1265, 8)